In [1]:
import pandas as pd
import tgt

#from pypinyin import pinyin, lazy_pinyin, Style
import glob
from googletrans import Translator
from pypinyin import pinyin, lazy_pinyin, Style


In [2]:
class sampa2ipa():
    def __init__(self):
        # 讀入 SAMPA
        self.sampadict = pd.read_excel('sampa_revised.xls')
        #print(sampadict)

    def get_result_len(self, x):
        if x[-2] == None:
            return 1
        elif x[-1] == None:
            return 2
        else:
            return 3
    
    def is_sampa_notation(self, text):
        sz = self.sampadict[self.sampadict['SAMPA'] == text].size
        return sz != 0

    def get_ipa(self, note1, note2, note3):
        """get_ipa takes SAMPA dictionary and 3 consecutive notations, and return an IPA Note list to be replaced,
           for example, when given SAMPA note1, note2, note3
           ('m', DONTCARE, DONTCARE), it will return [m, None, None]
           when given
           ('W', '@', 'n'), it will return ['ɥ', 'ɛ', 'n']

            if the result contains "_", it means that the two SAMPA notes needs to be merged into one IPA note
        """
        #print(">>>", note1, note2, note3)
        if note1 not in ['@', 'W', 'y', '&', 'w']:
            return [self.sampadict.loc[self.sampadict['SAMPA'] == note1]['IPA'].to_string(index = False).strip(), None, None]
        elif note1 == '@':
            return ['a', None, None]
        elif note1 == 'W':
            if note2 == '@' and note3 == 'n':
                return ['ɥ', 'ɛ', 'n']
            else:
                return ['ɥ', None, None]
        elif note1 == 'y':
            if note2 == 'i' and note3 == 'n':
                return ['i', '_', 'n']
            else:
                return ['j', None, None]
        elif note1 == '&':
            if note2 == 'N':
                return ['ə', 'ŋ', None]
            elif note2 == 'n':
                return ['ə', 'n', None]
            else:
                return ['ɤ', None, None]
        elif note1 == 'w':
            if note2 == '&' and note3 == 'N':
                return ['w','o','ŋ']
            elif note2 == 'u':
                return ['u', None, None]
            elif note2 == 'o':
                return ['ɔ', None, None]
            elif note2 == '>':
                return ['w','ɔ', None]
            else:
                return ['w', None, None]
        
    def get_cgvn(self, note1, note2, note3):
        """get_ipa takes SAMPA dictionary and 3 consecutive notations, and return an IPA Note list to be replaced,
           for example, when given SAMPA note1, note2, note3
           ('m', DONTCARE, DONTCARE), it will return [m, None, None]
           when given
           ('W', '@', 'n'), it will return ['ɥ', 'ɛ', 'n']

            if the result contains "_", it means that the two SAMPA notes needs to be merged into one IPA note
        """
        #print(">>>", note1, note2, note3)
        if note1 not in ['@', 'W', 'y', '&', 'w']:
            return [self.sampadict.loc[self.sampadict['SAMPA'] == note1]['CGVN'].to_string(index = False).strip(), None, None]
        elif note1 == '@':
            return ['V', None, None]
        elif note1 == 'W':
            if note2 == '@' and note3 == 'n':
                return ['G', 'V', 'N']
            else:
                return ['G', None, None]
        elif note1 == 'y':
            if note2 == 'i' and note3 == 'n':
                return ['V', 'N', None]
            else:
                return ['G', None, None]
        elif note1 == '&':
            if note2 == 'N' or note2 == 'n':
                return ['V', 'N', None]
            else:
                return ['V', None, None]
        elif note1 == 'w':
            if note2 == '&' and note3 == 'N':
                return ['G','V','N']
            elif note2 == 'u':
                return ['V', None, None]
            elif note2 == 'o':
                return ['V', None, None]
            elif note2 == '>':
                return ['G','V', None]
            else:
                return ['G', None, None]        
                                              
xam = sampa2ipa()

xl=xam.get_ipa('h','u', None)
print(xl)
xl=xam.get_cgvn('h','u', None)
x2=xam.get_cgvn('u', None, None)
print(xl, x2)

['x', None, None]
['C', None, None] ['V', None, None]


In [3]:
import os
import sys

# Suppress as many warnings as possible
# os.environ["TF_CPP_MIN_LOG_LEVEL"] = "3"
# from tensorflow.python.util import deprecation
# deprecation._PRINT_DEPRECATION_WARNINGS = False
# import tensorflow as tf
# tf.compat.v1.logging.set_verbosity(tf.compat.v1.logging.ERROR)

from ckiptagger import data_utils, construct_dictionary, WS, POS, NER

# Load model with GPU
ws = WS("./datackip", disable_cuda=False)
pos = POS("./datackip", disable_cuda=False)
ner = NER("./datackip", disable_cuda=False)

2024-02-25 10:34:28.513473: I tensorflow/core/platform/cpu_feature_guard.cc:193] This TensorFlow binary is optimized with oneAPI Deep Neural Network Library (oneDNN) to use the following CPU instructions in performance-critical operations:  AVX2 FMA
To enable them in other operations, rebuild TensorFlow with the appropriate compiler flags.
/home/chainwu/anaconda3/envs/tpanaly/lib/python3.8/site-packages/ckiptagger/model_ws.py:106: UserWarning: `tf.nn.rnn_cell.LSTMCell` is deprecated and will be removed in a future version. This class is equivalent as `tf.keras.layers.LSTMCell`, and will be replaced by that in Tensorflow 2.0.
  cell = tf.compat.v1.nn.rnn_cell.LSTMCell(hidden_d, name=name)
/home/chainwu/anaconda3/envs/tpanaly/lib/python3.8/site-packages/ckiptagger/model_pos.py:56: UserWarning: `tf.nn.rnn_cell.LSTMCell` is deprecated and will be removed in a future version. This class is equivalent as `tf.keras.layers.LSTMCell`, and will be replaced by that in Tensorflow 2.0.
  cell = tf.

In [4]:
def ckipx(s):
    sentence_list=[s]
    # Download data
    #data_utils.download_data("./")
    
    # Load model without GPU
    #ws = WS("./datackip")
    #pos = POS("./datackip")
    #ner = NER("./datackip")
    
    # Load model with GPU
#     ws = WS("./datackip", disable_cuda=False)
#     pos = POS("./datackip", disable_cuda=False)
#     ner = NER("./datackip", disable_cuda=False)
    
    # Create custom dictionary
    #word_to_weight = {
    #    "土地公": 1,
    #    "土地婆": 1,
    #    "公有": 2,
    #    "": 1,
    #    "來亂的": "啦",
    #    "緯來體育台": 1,
    #}
    #dictionary = construct_dictionary(word_to_weight)
    #print(dictionary)
    
    # Run WS-POS-NER pipeline
#     sentence_list = [
#         "太陽，",
#         '摩根大通資產管理公司就認為，當前中國股票的估值非常有吸引力，長期迴避中國股票的貝爾資產（Bell Asset Management）近期也罕見表態，考慮買入包括騰訊控股在內的中國大型科技公司。安本資產管理過去三個季度對中國保持中立，但現在開始看到中國股市價值。',
#         "傅達仁今將執行安樂死，卻突然爆出自己20年前遭緯來體育台封殺，他不懂自己哪裡得罪到電視台。",
#         "美國參議院針對今天總統布什所提名的勞工部長趙小蘭展開認可聽證會，預料她將會很順利通過參議院支持，成為該國有史以來第一位的華裔女性內閣成員。",
#         "",
#         "土地公有政策?？還是土地婆有政策。",
#         "… 你確定嗎… 不要再騙了……",
#         "最多容納59,000個人,或5.9萬人,再多就不行了.這是環評的結論.",
#         "科長說:1,坪數對人數為1:3。2,可以再增加。",
#     ]
    #word_sentence_list = ws(sentence_list)
    word_sentence_list = ws(sentence_list, sentence_segmentation=True)
    # word_sentence_list = ws(sentence_list, recommend_dictionary=dictionary)
    # word_sentence_list = ws(sentence_list, coerce_dictionary=dictionary)
    pos_sentence_list = pos(word_sentence_list)
    entity_sentence_list = ner(word_sentence_list, pos_sentence_list)
    
    # Release model
#     del ws
#     del pos
#     del ner
    
    # Show results
    def print_word_pos_sentence(word_sentence, pos_sentence):
        assert len(word_sentence) == len(pos_sentence)
        for word, pos in zip(word_sentence, pos_sentence):
            print(f"{word}({pos})", end="\u3000")
        print()
        return
    
#     for i, sentence in enumerate(sentence_list):
#         print()
#         print(f"'{sentence}'")
#         print_word_pos_sentence(word_sentence_list[i],  pos_sentence_list[i])
#         for entity in sorted(entity_sentence_list[i]):
#             print(entity)
    #[expression(item) for item in iterable]
    px =  [item for row in pos_sentence_list for item in row]
    return px #sentence_list

posx=ckipx('太陽')
print(posx)
#    sys.exit()
    

['Na']


In [8]:
class t13():
    def __print_tier__(self, t):
        return
        #for it in t.__iter__():
        #    print(it.text)

    def __foursounds__(self, c):
        #print(c)
        if 'ˋ' in c:
            return "51 "
        elif 'ˊ' in c:
            return "35 "
        elif 'ˇ' in c:
            return "21 "
        elif '˙' in c:
            return "0 "
        else:
            return "55 "
        
    def chinese(self, namestr):
        t=tgt.IntervalTier(start_time=0, end_time=10, name=namestr)
        wt = self.template.get_tier_by_name("word")
        wti = wt.__iter__()
        #print(wti)
        for a in wti:
            #print(a.text)
            na = tgt.Annotation(a.start_time, a.end_time, text=a.text)
            t.add_annotation(na)
            
        naa = tgt.Annotation(na.end_time, 10, text='sp')
        t.add_annotation(naa)
          
        self.grid.add_tier(t)

    def cterm(self, namestr):
        t=tgt.IntervalTier(start_time=0, end_time=10, name=namestr)
        wt = self.template.get_tier_by_name("word")
        #print(wti)
        objlen = len(wt._objects)
        idx = 0
        while idx < objlen-1:
            a1 = wt._objects[idx]
            a2 = wt._objects[idx+1]
            print(idx, a1, a2)

            if a1.text != 'sp' and a2.text != 'sp':
                na = tgt.Annotation(a1.start_time, a2.end_time, text=a1.text+a2.text)
                t.add_annotation(na)
                idx=idx+2
            else:
                na = tgt.Annotation(a1.start_time, a1.end_time, text=a1.text)
                t.add_annotation(na)
                idx=idx+1
                
        #print(na)
        na = tgt.Annotation(wt._objects[idx].start_time, 10, text=wt._objects[idx].text)
        t.add_annotation(na)

        self.__print_tier__(t)
        self.grid.add_tier(t)
        
    def english(self, namestr):
        t=tgt.IntervalTier(start_time=0, end_time=10, name=namestr)
        wt = self.template.get_tier_by_name("word")
        wt = self.grid.get_tier_by_name('chinese-t')
        wti = wt.__iter__()
        #print(wti)
        for a in wti:
            if a.text != 'sp':
                tr = self.translator.translate(a.text, dest = 'en')
                na = tgt.Annotation(a.start_time, a.end_time, text=tr.text)
            else:
                na = tgt.Annotation(a.start_time, a.end_time, text=a.text)

            t.add_annotation(na)

        self.__print_tier__(t)        
        self.grid.add_tier(t)

    def pos(self, namestr):
        t=tgt.IntervalTier(start_time=0, end_time=10, name=namestr)
        ct=self.grid.get_tier_by_name("chinese-t")
        wti = ct.__iter__()
        #print(wti)
        for a in wti:
            if a.text != 'sp':
                posx = ckipx(a.text)
                nposx = [x for xs in posx for x in xs]
                posxx= ' '.join(nposx)
                na = tgt.Annotation(a.start_time, a.end_time, text=posxx)
            else:
                na = tgt.Annotation(a.start_time, a.end_time, text=a.text)

            t.add_annotation(na)
        
        self.__print_tier__(t)        
        self.grid.add_tier(t)


    def ipa(self, namestr):
        self.xam = sampa2ipa()
        t=tgt.IntervalTier(start_time=0, end_time=10, name=namestr)
        wordtier = self.template.get_tier_by_name("word")
        phonetier = self.template.get_tier_by_name("phone")

        for wann in wordtier._objects:
            annlist = phonetier.get_annotations_between_timepoints(wann.start_time, wann.end_time)
            plen = len(annlist)
            i = 0
            while i < plen :
                ann = annlist[i]
                #print(i, ":", ann.text)

                if ann.text == "sp":
                    newann = tgt.core.Annotation(ann.start_time, ann.end_time, 'sp')
                    t.add_annotation(newann)
                    i = i + 1
                    continue

                if not xam.is_sampa_notation(ann.text):
                    newann = tgt.core.Annotation(ann.start_time, ann.end_time, "")
                    i = i + 1
                    continue

                if i < plen - 2:
                    annplus1 = annlist[i+1]
                    annplus2 = annlist[i+2]
                    arg = ann.text
                    arg1 = annplus1.text
                    arg2 = annplus2.text
                elif i == plen - 2:
                    annplus1 = annlist[i+1]
                    arg = ann.text
                    arg1 = annplus1.text
                    arg2 = None
                else:
                    arg = ann.text
                    arg1 = None
                    arg2 = None                

                resultlist = xam.get_ipa(arg, arg1, arg2)
                results_len = xam.get_result_len(resultlist)
                #print(resultlist)

                for j in range(len(resultlist)):
                    if resultlist[j] == None:
                        continue
                    if resultlist[j] == '_':
                        i = i + 1
                        newann = tgt.core.Annotation(prevann.start_time, annplus1.end_time, prevann.text)
                        t.delete_annotation_by_start_time(prevann.start_time)
                        t.add_annotation(newann)
                    else:
                        ann = annlist[i]
                        newann = tgt.core.Annotation(ann.start_time, ann.end_time, resultlist[j])
                        i = i + 1
                        prevann = newann   
                        t.add_annotation(newann)
        
        newann._set_end_time(10)

        self.__print_tier__(t)
        self.grid.add_tier(t)
        
    def syllables(self, namestr):
        t=tgt.IntervalTier(start_time=0, end_time=10, name=namestr)
        wordtier = self.template.get_tier_by_name("word")
        phonetier = self.template.get_tier_by_name("phone")

        for wann in wordtier._objects:
            annlist = phonetier.get_annotations_between_timepoints(wann.start_time, wann.end_time)
            plen = len(annlist)
            i = 0
            while i < plen :
                ann = annlist[i]
                #print(i, ":", ann.text)

                if ann.text == "sp":
                    newann = tgt.core.Annotation(ann.start_time, ann.end_time, 'sp')
                    t.add_annotation(newann)
                    i = i + 1
                    continue

                if not xam.is_sampa_notation(ann.text):
                    newann = tgt.core.Annotation(ann.start_time, ann.end_time, "")
                    i = i + 1
                    continue

                if i < plen - 2:
                    annplus1 = annlist[i+1]
                    annplus2 = annlist[i+2]
                    arg = ann.text
                    arg1 = annplus1.text
                    arg2 = annplus2.text
                elif i == plen - 2:
                    annplus1 = annlist[i+1]
                    arg = ann.text
                    arg1 = annplus1.text
                    arg2 = None
                else:
                    arg = ann.text
                    arg1 = None
                    arg2 = None                

                resultlist = xam.get_cgvn(arg, arg1, arg2)
                results_len = xam.get_result_len(resultlist)
                #print(resultlist)

                for j in range(len(resultlist)):
                    if resultlist[j] == None:
                        continue
                    if resultlist[j] == '_':
                        i = i + 1
                        newann = tgt.core.Annotation(prevann.start_time, annplus1.end_time, prevann.text)
                        t.delete_annotation_by_start_time(prevann.start_time)
                        t.add_annotation(newann)
                    else:
                        ann = annlist[i]
                        newann = tgt.core.Annotation(ann.start_time, ann.end_time, resultlist[j])
                        i = i + 1
                        prevann = newann   
                        t.add_annotation(newann)
                        
        newann._set_end_time(10)
        self.__print_tier__(t)
        self.grid.add_tier(t)
        
    def tone(self, namestr):
        t=tgt.IntervalTier(start_time=0, end_time=10, name=namestr)
        wt = self.template.get_tier_by_name("word")
        pt = self.template.get_tier_by_name("phone")

        for wann in wt._objects:
            if wann.text != 'sp':
                ccl = pinyin(wann.text, style=Style.BOPOMOFO)
                #print(ccl[0][0][-1])
                dcl = self.__foursounds__(ccl[0][0][-1])
                #print(ccl, dcl)
            else:
                dcl='sp'
                
            newann = tgt.core.Annotation(wann.start_time, wann.end_time, dcl)
            t.add_annotation(newann)
        
        newann._set_end_time(10)
        self.__print_tier__(t)
        self.grid.add_tier(t)    
    
    def type_tier(self, namestr):
        t=tgt.IntervalTier(start_time=0, end_time=10, name=namestr)
        newann = tgt.core.Annotation(t.start_time, t.end_time, 'NA')
        t.add_annotation(newann)
            
        self.grid.add_tier(t)            

    def participant(self, namestr,part):
        t=tgt.IntervalTier(start_time=0, end_time=10, name=namestr)
        newann = tgt.core.Annotation(t.start_time, t.end_time, part)
        t.add_annotation(newann)
        
        self.grid.add_tier(t)

        return
    
    def export_textgrid(self, fn):
         tgt.io.write_to_file(self.grid, fn+".13", format="long")
            
    def __init__(self, tm, part):
        self.newfn="data/NSB_I_PicNaming/Standard_Pic-naming/"+tm+".textgrid"
        self.xam = sampa2ipa()
        self.translator = Translator()

        self.picfn=""
        self.template = tgt.io.read_textgrid(self.newfn, encoding='utf-8')
        self.grid = tgt.TextGrid()
        #self.sampatier=self.template.get_tier_by_name("phone")
        #self.chitier=self.template.get_tier_by_name("word")
        self.fn=part
        
        self.chinese('chinese-x')
        self.cterm('chinese-t')
        self.english('english-t')
        self.pos('pos-t')
        self.ipa('ipa-t')
        self.syllables('syllable-t')
        self.tone('tone-t')
        self.cterm('chinese-s')
        self.english('english-s')
        self.pos('pos-s')
        self.ipa('ipa-s')
        self.syllables('syllable-s')
        self.tone('tone-s')
        self.type_tier('type')
        self.participant('participant', part)
        
        self.grid.delete_tier('chinese-x')
        
x=t13("太陽","m1324")
x.export_textgrid("./xyz")


0 Interval(0.0125, 1.0125, "sp") Interval(1.0125, 1.4725, "太")
1 Interval(1.0125, 1.4725, "太") Interval(1.4725, 1.9425, "陽")
0 Interval(0.0125, 1.0125, "sp") Interval(1.0125, 1.4725, "太")
1 Interval(1.0125, 1.4725, "太") Interval(1.4725, 1.9425, "陽")


In [10]:
def main(txtgridr):
    #Construct tier contents
    #fl=glob.glob("data/**/Standard_Pic-naming/*.wav", recursive=True)
    fl=glob.glob("data/**/Standard_Pic-naming/*.wav", recursive=True)
    for f in fl:
        fn = f[:-4]
        cn=fn.split("/")[-1]
        print(cn, fn, end=" ")        
        t13txtgrid = t13(cn, 'NAX')
        t13txtgrid.export_textgrid(fn)
        
        
main("./data/NSB_I_PicNaming/Standard_Pic-naming")

葡萄 data/NSB_I_PicNaming/Standard_Pic-naming/葡萄 0 Interval(0.0125, 0.2325, "sp") Interval(0.2325, 0.5625, "葡")
1 Interval(0.2325, 0.5625, "葡") Interval(0.5625, 1.0925, "萄")
0 Interval(0.0125, 0.2325, "sp") Interval(0.2325, 0.5625, "葡")
1 Interval(0.2325, 0.5625, "葡") Interval(0.5625, 1.0925, "萄")
貓 data/NSB_I_PicNaming/Standard_Pic-naming/貓 0 Interval(0.0125, 0.9724999999999999, "sp") Interval(0.9724999999999999, 1.5225, "貓")
1 Interval(0.9724999999999999, 1.5225, "貓") Interval(1.5225, 2.2525000000000004, "sp")
0 Interval(0.0125, 0.9724999999999999, "sp") Interval(0.9724999999999999, 1.5225, "貓")
1 Interval(0.9724999999999999, 1.5225, "貓") Interval(1.5225, 2.2525000000000004, "sp")
蠟筆 data/NSB_I_PicNaming/Standard_Pic-naming/蠟筆 0 Interval(0.0125, 0.8524999999999999, "sp") Interval(0.8524999999999999, 1.3125, "蠟")
1 Interval(0.8524999999999999, 1.3125, "蠟") Interval(1.3125, 1.6424999999999998, "筆")
0 Interval(0.0125, 0.8524999999999999, "sp") Interval(0.8524999999999999, 1.3125, "蠟")
1 I